<h1 align="center"> TradePro Strategy 1 </h1> 

<h2 align="center"> Blacksheep Capital | Bryan Marty, CFA<h2>





---
# TradePro Strategy:(Traditional Approach)

### LOGIC:

    - Go long when current price > 8 EMA > 14 EMA > 50 EMA and when the stochastic K line moves above the stochastic d line == Buy signal.  
    - Profit Target: 2 * ATR
    - Stop Loss: 3*ATR
    
    
This is a scalping style strategy.Tested in one hour time bars proved 1.5:1 Risk/Reward for 100 trades in 6 Months time horizon In the EUR/USD Market
    
Our first interation will use Daily Bars and not hourly Bars until we can leverage Websockets and BinanceAPI sufficiently at that time frame. 
    
### Why TradePro:

TradePro was elected as the first strategy due to the fact it has shown sufficient robustness in the FX market which exhibits similarites to digital assets with regards to volatility.  In addtion, given the idea, it is a baseline strategy that exhibits low corraltion to the higher "market" beta strategy that is Discretionarily provided by Orca 1. Furthermore, given parameter optimization we could elect to keep or discard given we find another similar stratey with superior mathematical properties and expression of the original Scalping Idea

---
## Scalping Strategy Definition:

Scalping is a trading strategy geared towards profiting from minor price changes in a stock's price. Traders who implement this strategy place anywhere from 10 to a few hundred trades in a single day with the belief that small moves in stock price are easier to catch than large ones; traders who implement this strategy are known as scalpers. Many small profits can easily compound into large gains if a strict exit strategy is used to prevent large losses.

### Basics of Scalping
Scalping utilizes larger position sizes for smaller price gains in the smallest period of holding time. It is performed intraday. The main goal is to buy or sell a number of shares at the bid—or ask—price and then quickly sell them a few cents higher or lower for a profit. The holding times can vary from seconds to minutes, and in some cases up to several hours. The position is closed before the end of the total market trading session, which can extend to 8 p.m. EST.

# Parameter Definitions
---
### Exponential Moving Average:

The exponential moving average (EMA) is a technical chart indicator that tracks the price of an investment (like a stock or commodity) over time. The EMA is atype of weighted moving average (WMA) that gives more weighting or importance to recent price data.
    
### Average True Range: 

The average true range (ATR) is a market volatility indicator used in technical analysis.  It is typically derived from the 14-day simple moving average of a series of true range indicators.
The ATR was originally developed for use in commodities markets but has since been applied to all types of securities.

The true range indicator is taken as the greatest of the following: current high less the current low; the absolute value of the current high less the previous close;and the absolute value of the current low less the previous close. The ATR is then a moving average, generally using 14 days, of the true ranges.
    

https://www.investopedia.com/terms/a/atr.asp#:~:text=The%20average%20true%20range%20(ATR)%20is%20a%20market%20volatility%20indicator,to%20all%20types%20of%20securities.

### Stochastic RSI %

The StochRSI oscillator was developed to take advantage of both momentum indicators in order to create a more sensitive indicator that is attuned to a specific security's historical performance rather than a generalized analysis of price change.
https://www.investopedia.com/terms/s/stochrsi.asp


# Backtest & Results 

- Today we are using backtesting.py to backtest and to optimize this strategy.  
- Full Documentation found here:
 
 https://kernc.github.io/backtesting.py/doc/backtesting/#gsc.tab=0




### Backtesting Results and Code below

insample


In [2]:
# Import Libraries
from binance.client import Client
#from keys_secrets import Binance_API#
import numpy as np
import pandas as pd
import yfinance as yf
import ta
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from datetime import date, timedelta, datetime
import seaborn as sns
import matplotlib.pyplot as plt
import math
from BinanceData import BinanceData

# [georgi]: Added parent dir to path to make it possible to import BinanceData and BinanceTrader
import sys
sys.path.append('../')
from BinanceData import BinanceData
#from BinanceTrader import BinanceTrader : This is not going to use our TraderClass in the public repo

client = Client()

In [3]:
#instantiate data
data = BinanceData()

btc_daily = data.getdailydata('BTC', 'USDT',  '2019-01-01') # from jan 1 2019
btc_hourly = data.gethourdata('BTC', 'USDT', 4320) # 4 months
btc_minute = data.getminutedata('BTC', 'USDT', 10080) # 2 weeks


In [4]:
#TradePro Class
class TradePro(Strategy):
    
    """Read the Bactesting CLASS for details but 
    essentially we do not need Super init in this class
    and can use the parent class directly as such.
    
    The Purpose of this is to show you how to use another Type of backtestor
    
    Backtester.py is a great backtestor that provides appealing visuals however in my opinion it is 
    
    significanclty lacking analytical insight like VaR, ES, Clustered MDA, ect.  That said this is just to show someone how 
    
    to use it in the most general form. """
    
    window_ema1 = 8
    window_ema2 = 14
    window_ema3 = 50
    
    
    def init(self):
        # use these three guys in the classes below
        close = self.data.Close
        high = self.data.High
        low = self.data.Low
        
        # using oop to use the ta library and bactetsing class together!
        self.stoch_k = self.I(ta.momentum.stochrsi_k, pd.Series(close))
        self.stoch_d = self.I(ta.momentum.stochrsi_d, pd.Series(close))
        self.EMA_8 = self.I(ta.trend.ema_indicator, pd.Series(close), window = self.window_ema1)
        self.EMA_14 = self.I(ta.trend.ema_indicator, pd.Series(close), window = self.window_ema2)
        self.EMA_50 = self.I(ta.trend.ema_indicator, pd.Series(close), window = self.window_ema3)
        
        # The indicator provide an indication of the degree of price volatility. 
        # Strong moves, in either direction, are often accompanied by large ranges,
        # or large True Ranges.
        self.atr = self.I(ta.volatility.average_true_range, pd.Series(high), pd.Series(low), pd.Series(close))
        
    def next(self):
        """Strategy logic"""
        price = self.data.Close[-1]
        # check for stochastic_rsi_kline crossing over stochastic rsi d line and price beating ema8, ema14, ema 50
        if (crossover(self.stoch_k, self.stoch_d) and 
            price > self.EMA_8 and 
            self.EMA_8 > self.EMA_14 and
            self.EMA_14 > self.EMA_50):
            sl = price - self.atr*3 # stop loss if 3 x biger than 14 day window of high/low/close volatility.
            tp = price + self.atr*2 # not getting greedy we take 2 times the average true vol. (WD COULD UES GARCH(1,1) HERE AS WELL)
            
            #I am commenting out the self.buy method call and adding the signal, tp price and sl price to self.data dataframe
            self.buy(sl = sl, tp = tp)


In [5]:
# this is the "slowest" manner as well with trades happeing t+1 days after the events. 
bt_btc = Backtest(btc_daily, TradePro
                  , cash=10_000, commission=.001)    
print(bt_btc.run())
bt_btc.plot()

<ipython-input-5-81bb41573198>:2: UserWarning: Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-134).
  bt_btc = Backtest(btc_daily, TradePro


Start                     2019-01-01 00:00:00
End                       2022-09-14 00:00:00
Duration                   1352 days 00:00:00
Exposure Time [%]                   25.720621
Equity Final [$]                 16932.691673
Equity Peak [$]                  16932.691673
Return [%]                          69.326917
Buy & Hold Return [%]              430.935652
Return (Ann.) [%]                   15.266635
Volatility (Ann.) [%]               26.787482
Sharpe Ratio                         0.569917
Sortino Ratio                        1.003903
Calmar Ratio                         0.467375
Max. Drawdown [%]                  -32.664661
Avg. Drawdown [%]                    -5.38248
Max. Drawdown Duration      398 days 00:00:00
Avg. Drawdown Duration       36 days 00:00:00
# Trades                                   22
Win Rate [%]                        81.818182
Best Trade [%]                      12.389428
Worst Trade [%]                    -24.683956
Avg. Trade [%]                    

Row(id='1654', ...)

# Hourly

- This doesn't Trade and thats a good thing.  It is essentially not getting a buy signal.
- One could put like 300000000 hours for the look back to get a more comprehensive view of how hourly perorms. 
- Scalping strategies were designed to perform better in hourly time sampling




In [6]:
bt_btc = Backtest(btc_hourly, TradePro
                  , cash=10_000, commission=.001)    
print(bt_btc.run())
bt_btc.plot()

<ipython-input-6-84ff6b8da385>:1: UserWarning: Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-134).
  bt_btc = Backtest(btc_hourly, TradePro


Start                     2022-03-18 01:00:00
End                       2022-09-14 00:00:00
Duration                    179 days 23:00:00
Exposure Time [%]                         0.0
Equity Final [$]                      10000.0
Equity Peak [$]                       10000.0
Return [%]                                0.0
Buy & Hold Return [%]              -50.282252
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     0.0
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              NaN
Max. Drawdown [%]                        -0.0
Avg. Drawdown [%]                         NaN
Max. Drawdown Duration                    NaN
Avg. Drawdown Duration                    NaN
# Trades                                    0
Win Rate [%]                              NaN
Best Trade [%]                            NaN
Worst Trade [%]                           NaN
Avg. Trade [%]                    

Row(id='2525', ...)

# Minute data couldn't handle the strategy:


---


In [8]:
# bt_btc = Backtest(btc_minute, TradePro
#                   , cash=10_000, commission=.001)    
# print(bt_btc.run())
# bt_btc.plot()
# Produces ValueError: Long orders require: SL (18772.74084110273) < LIMIT (18816.888089999997) < TP (18814.989439264846) suggesting in minute intervals
# we need ot re-configure the sl, and tp. 

## Example of Optimization 

- This is a nice feature of backtesting.py.
- When you instantiate your variables as I did at onset (not hard code them) you will be able to opmize as shown below
- You can optimize ANY rowname of the instance.run() method so long as you put the parameters you want optimized as class variables.

In [9]:
bt = Backtest(btc_daily, TradePro, cash = 100000, commission= 0.002, exclusive_orders = True)

stats = bt.optimize(window_ema1 = range(5, 200, 5), # this should be higher range than lower bound but I'm demonstrating the constraint function here
                   window_ema2 = range(5, 200, 5),
                   window_ema3 = range(5,200, 5),
                   maximize = "Sortino Ratio",
                   constraint = lambda param: param.window_ema1  < param.window_ema2 < param.window_ema3, # this is a constraint you can pass. 
                   max_tries=700)# This is showing us how to put a constraint on the . Max_tries: so we don't overfit.
print(stats)
print(bt.plot())


  0%|          | 0/19 [00:00<?, ?it/s]

Start                     2019-01-01 00:00:00
End                       2022-09-14 00:00:00
Duration                   1352 days 00:00:00
Exposure Time [%]                   35.254989
Equity Final [$]                173390.621344
Equity Peak [$]                 204713.750213
Return [%]                          73.390621
Buy & Hold Return [%]              430.935652
Return (Ann.) [%]                   16.006453
Volatility (Ann.) [%]               41.665147
Sharpe Ratio                         0.384169
Sortino Ratio                         0.63796
Calmar Ratio                         0.349362
Max. Drawdown [%]                  -45.816266
Avg. Drawdown [%]                   -7.200366
Max. Drawdown Duration      540 days 00:00:00
Avg. Drawdown Duration       54 days 00:00:00
# Trades                                   69
Win Rate [%]                        65.217391
Best Trade [%]                      17.934569
Worst Trade [%]                    -22.551447
Avg. Trade [%]                    

Row(id='3467', ...)
